In [65]:
import fitz

def extract_text_from_pdf(pdf_path):
    text = ""
    with fitz.open(pdf_path) as doc:
        for page in doc:
            text += page.get_text()
    return text

pdf_path = "files/labor_code.pdf"
text = extract_text_from_pdf(pdf_path)

In [66]:
text

'AZƏRBAYCAN RESPUBLİKASININ ƏMƏK MƏCƏLLƏSİ\n \nAzərbaycan Respublikası Konstitusiyasının 35-ci maddəsinə əsasən əmək fərdi və ictimai rifahın əsasıdır. Hər\nkəsin əməyə olan qabiliyyəti əsasında sərbəst surətdə özünə fəaliyyət növü, peşə, məşğuliyyət və iş yeri seçmək\nhüququ vardır.\nAzərbaycan Respublikasının Əmək Məcəlləsində:\nmüvafiq hüquq normaları ilə əmək münasibətlərində işçilərin və işəgötürənlərin əmək, sosial, iqtisadi\nhüquqları və bu hüquqlarla bağlı müvafiq təminatların minimum səviyyəsi;\nAzərbaycan Respublikası Konstitusiyasının ikinci bölməsində nəzərdə tutulan əmək, istirahət, təhlükəsiz və\nsağlam şəraitdə işləmək hüququnun, habelə digər əsas insan hüquqlarının və azadlıqlarının təmin edilməsi\nprinsipləri və qaydaları;\nAzərbaycan Respublikasının bağladığı və ya tərəfdar çıxdığı beynəlxalq müqavilələrə, Beynəlxalq Əmək\nTəşkilatının konvensiyalarına və digər beynəlxalq hüquq normalarına uyğun olaraq əmək münasibətlərinin\nyaranması, dəyişdirilməsi, onlara xitam ver

In [67]:
with open("files/labor_code.txt", "w", encoding="utf-8") as f:
    f.write(text)

print("PDF text extraction complete.")

PDF text extraction complete.


In [68]:
import re

pattern = r"\n.+fəsil\n"

matches = [match.start() for match in re.finditer(pattern, text)]

In [69]:
pattern = r"\nƏLAVƏLƏR\n"

match = re.search(pattern, text)

end_of_txt = match.start() if match else None
end_of_txt

344402

In [70]:
import re
import json

all_articles = []
article_index = 0  # Unique index for each maddə

for i in range(len(matches)):
    start = matches[i]
    end = matches[i+1] if i+1<len(matches) else end_of_txt
    
    fesil = text[start: end]
    
    pattern = r"\n.+fəsil\n"
    match = re.search(pattern, fesil)
    fesil_heading = match.group().strip() if match else "N/A" # metadata
    
    pattern = r"fəsil(.*?)Maddə"
    match = re.search(pattern, fesil, re.DOTALL | re.IGNORECASE)
    fesil_name = match.group(1).strip().replace('\n', " ") if match else "N/A" # metadata
    
    pattern = r"Maddə\s+(\d|-)+\."
    madde_matches = [match.start() for match in re.finditer(pattern, fesil)]
    
    for j in range(len(madde_matches)):
        start = madde_matches[j]
        end = madde_matches[j+1] if j+1<len(madde_matches) else len(fesil)
        
        madde = fesil[start: end]
        
        pattern = r"Maddə\s+(\d|-)+\."
        match = re.search(pattern, madde)
        madde_heading = match.group().strip().replace('\n', "") if match else "N/A" # metadata
        
        pattern = r"Maddə\s+(\d|-)+\.(.*?)\n\s"
        match = re.search(pattern, madde, re.DOTALL | re.IGNORECASE)
        madde_name = match.group(2).strip().replace('\n', " ") if match else "N/A" # metadata
        
        content_start = match.end() if match else 0
        madde_context = madde[content_start:].strip() # context 
        
        all_articles.append({
            "index": article_index,
            "chapter_heading": fesil_heading,
            "chapter_title": fesil_name,
            "article_heading": madde_heading,
            "article_title": madde_name,
            "article_body": madde_context
        })

        article_index += 1

with open("articles.json", "w", encoding="utf-8") as f:
    json.dump(all_articles, f, ensure_ascii=False, indent=2)

print(f"Saved {len(all_articles)} articles to articles.json")       

Saved 325 articles to articles.json


In [ ]:
rag_contexts = []

for i in all_articles:
    context = f"{i['chapter_heading']} {i['chapter_title']}\n\n{i['article_heading']} {i['article_title']}\n\n{i['article_body']}"
    rag_contexts.append(context)

In [72]:
rag_contexts

['Birinci fəsil Əsas müddəalar\n\nMaddə 1. Azərbaycan Respublikasının əmək qanunvericiliyi sisteminə daxil olan normativ hüquqi aktlar\n\nAzərbaycan Respublikasının əmək qanunvericiliyi sistemi:\nbu Məcəllədən;\nAzərbaycan Respublikasının müvafiq qanunlarından;\nmüvafiq icra hakimiyyəti orqanlarının səlahiyyəti çərçivəsində qəbul etdiyi normativ hüquqi aktlardan;\nəmək, sosial-iqtisadi məsələlərlə əlaqədar Azərbaycan Respublikasının bağladığı və ya tərəfdar çıxdığı\nbeynəlxalq müqavilələrdən ibarətdir.',
 'Birinci fəsil Əsas müddəalar\n\nMaddə 2. Azərbaycan Respublikası Əmək Məcəlləsinin vəzifələri və prinsipləri\n\n1. Azərbaycan Respublikasının Əmək Məcəlləsi işçilərlə işəgötürənlər arasında yaranan əmək münasibətlərini,\nhabelə onlarla müvafiq dövlət hakimiyyəti orqanları, hüquqi şəxslər arasında həmin münasibətlərdən törəyən digər\nhüquq münasibətlərini tənzim edir.\n2. Azərbaycan Respublikasının Əmək Məcəlləsi fiziki şəxslərin əmək hüquqlarının və bu hüquqların həyata\nkeçirilməsin

In [73]:
from components.embedder import BGEEmbedder
from components.vector_db import FaissVectorDB

db = FaissVectorDB(dim=1024)
embedder = BGEEmbedder()

embeddings = embedder.embed(rag_contexts)
print(len(rag_contexts))
print(len(embeddings))

db.upload(embeddings)

db.save()

325
325


In [74]:
query = embedder.embed_sentence("İddianın məbləğində və onun təmin olunmasında məhdudiyyətin olmaması")
query = query.reshape(1, -1)

db.search(query_embedding=query, top_k=3)

[{'index': 305, 'distance': 0.649273157119751},
 {'index': 188, 'distance': 0.9579514265060425},
 {'index': 183, 'distance': 0.9814552068710327}]